In [2]:
import sys
import pandas as pd
import numpy as np
import os
import time
import clr

import win32com.client
"""
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.GenericMotorCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\ThorLabs.MotionControl.KCube.DCServoCLI.dll")
from Thorlabs.MotionControl.DeviceManagerCLI import *
from Thorlabs.MotionControl.GenericMotorCLI import *
from Thorlabs.MotionControl.KCube.DCServoCLI import *
from System import Decimal
"""


sys.path.append("../External_Instruments")
sys.path.append("../Multiplexer_Board")
#sys.path.append("../Goniometer")
sys.path.append("../Measurement_Cycles")




import External_Instruments, PCB_Board, Measurement#,Goniometer

In [3]:
import win32com.client



def detect_setups():
    df_available_setups=External_Instruments.recognize_setup()
    wmi = win32com.client.GetObject("winmgmts:")
    for usb in wmi.InstancesOf("Win32_SerialPort"):
        if("Arduino" in usb.Description):
            new_row={"Setup_Name" : "Arduino", "GPIB_ADDRESS": usb.DeviceID, "IDN_RESPONSE":"?"}
            #df_available_setups = pd.concat([df_available_setups,new_row],ignore_index=True)
            new_row_df = pd.DataFrame([new_row])  # Convert new_row to a DataFrame
            df_available_setups = pd.concat([df_available_setups, new_row_df], ignore_index=True)


        else:
            new_row={"Setup_Name" : "Cannon Board", "GPIB_ADDRESS": usb.DeviceID, "IDN_RESPONSE":"?"}
            new_row_df = pd.DataFrame([new_row])  # Convert new_row to a DataFrame
            df_available_setups = pd.concat([df_available_setups, new_row_df], ignore_index=True)


    
    new_row={"Setup_Name" : "Goniometer", "GPIB_ADDRESS": "?", "IDN_RESPONSE":"?"}
    new_row_df = pd.DataFrame([new_row])  # Convert new_row to a DataFrame
    df_available_setups = pd.concat([df_available_setups, new_row_df], ignore_index=True)


    
    return df_available_setups

df=detect_setups()
print(df)


c:\Users\Pedro\anaconda3\lib\site-packages\pyvisa\ctwrapper\highlevel.py:226: VisaIOWarning: VI_WARN_CONFIG_NLOADED (1073676407): The specified configuration either does not exist or could not be loaded. VISA-specified defaults will be used.
  return self.handle_return_value(session, ret_value)  # type: ignore


      Setup_Name GPIB_ADDRESS IDN_RESPONSE           Type        Nickname
0  KEITHLEY 2401          NaN          NaN    Sourcemeter  2K Sourcemeter
1          KEPCO          NaN          NaN  Magnetic coil    400 Oe Coils
2          KEPCO          NaN          NaN  Magnetic coil    140 Oe Coils
3        HP6032A          NaN          NaN  Magnetic coil        2K Coils
4     Goniometer            ?            ?            NaN             NaN


In [4]:
import re
import tkinter as tk
from tkinter import messagebox

def parse_input(input_string):
    # Use regular expression to extract numbers (including decimals and exponents) from the input string
    matches = re.findall(r"-?\d+(?:\.\d*)?(?:e[-+]?\d+)?", input_string)
    # Convert the matched strings to floats
    numbers = [float(match) for match in matches]
    return numbers

def generate_values(range_str, increment_str=""):
    ranges = parse_input(range_str)
    increments = parse_input(increment_str)
    
    # Error checking
    if any(step <= 0 for step in increments):
        messagebox.showerror("Error","Increment values should be positive.")
        return
    if len(increments) != len(ranges) - 1:
        messagebox.showerror("Error","The length of increments should be one less than the length of ranges.")
        return
    
    values = []
    increment_idx = 0
    
    # Go through the range pairs and their respective increment values
    for i in range(len(ranges)-1):
        start = ranges[i]
        end = ranges[i+1]
        increment = increments[increment_idx]
        increment_idx += 1
        
        # Generate values for this range
        if start > end:
            while start >= end:
                values.append(start)
                start -= increment
        else:
            while start <= end:
                values.append(start)
                start += increment
                
    

    # Append the final value from the ranges
    values.append(ranges[-1])

    # Remove duplicates
    final_values = []
    [final_values.append(x) for x in values if x not in final_values]

    return final_values

print(parse_input("1,2"))
""" 
# Test
range_string = "-500"
increment_string = "-1"
print(generate_values(range_string, increment_string)) """


[1.0, 2.0]


' \n# Test\nrange_string = "-500"\nincrement_string = "-1"\nprint(generate_values(range_string, increment_string)) '

In [5]:
import tkinter as tk
from tkinter import ttk, filedialog

def Measurements_available():
    df=pd.DataFrame({ "Measurement_name" : ["Current_Source - 2PP", "Current_Source - Angular - 2PP","Beatriz likes acid"],
                             "Compatible Current Source"   : [["2K Sourcemeter"], ["2K Sourcemeter"], ["2K Sourcemeter"]],
                             "Compatible Voltage Source"   : [["2K Sourcemeter"], ["2K Sourcemeter"], ["2K Sourcemeter"]],
                             "Compatible Magnetic Coil": [["400 Oe Coils", "140 Oe Coils", "2K Coils"],["400 Oe Coils", "140 Oe Coils", "2K Coils"], ["140 Oe Coils"]],
                             "Compatible CannonBoard": [["Cannon Board"],["Cannon Board"],["Cannon Board"]],
                             "Compatible Goniometer": [["None"],["Goniometer"],["None"]],
                             "Compatible Probe Number" : [[2], [2],[2]],
                             "Compatible Measurement Type" : [["Current Source Measurement"],["Current Source Measurement"],["Current Source Measurement"]]})
    return df

def Start_Recipe(recipe_name,**kwargs):
    if recipe_name=="Current_Source - 2PP":
        Measurement.Current_Source_2PP(**kwargs)
    if recipe_name=="Current_Source - Angular - 2PP":
        Measurement.Current_Source_Angular_2PP(**kwargs)
    if recipe_name=="Beatriz likes acid":
        Measurement.Beatriz_likes_acid(**kwargs)


class InstrumentControlInterface(tk.Tk):

    def __init__(self):
        super().__init__()

        self.title("Instrument Control Interface")
        self.geometry("630x700")  # Adjusted size slightly to accommodate the changes

        # Main frame
        self.main_frame = ttk.Frame(self, padding="10")
        self.main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        # Experimental setups frame
        self.experimental_frame = ttk.LabelFrame(self.main_frame, text="Experimental Setups", padding="10")

        self._build_interface()
        self.load_default_values()

        self.save_directory = ""  # Variable to store the selected directory path

        self.update_comboboxes()

    def load_default_values(self):
        self.magnetic_field_range_entry.insert(0,"-100,100")
        self.increment_steps_entry.insert(0,"10")
        self.pin_number_combobox.set(24)
        self.probe_number_combobox.set(2)
        self.current_value_entry.insert(0,"1e-5")
        self.voltage_limit_entry.insert(0,"10")
        self.measurement_type_combobox.set("Current Source Measurement")
        self.channel_list_entry.insert(0,"3,4")
        #self.measurement_recipe_combobox.set("")
       



        
    def _build_interface(self):
        self._create_entries()
        self._create_comboboxes()
        self._create_buttons()
        self._create_labels()
        self._create_checkboxes()
        self.experimental_frame.grid(row=12, column=0, columnspan=2, sticky=(tk.W, tk.E, tk.N, tk.S), pady=10)

    def _create_entries(self):
        self.magnetic_field_range_entry = self._create_entry("Magnetic Field Range:", 0)
        self.increment_steps_entry = self._create_entry("Increment Steps:", 1)
        self.current_value_entry = self._create_entry("Current Value:", 5)
        self.voltage_limit_entry = self._create_entry("Voltage Limit:", 6)
        self.channel_list_entry = self._create_entry("Channel List:", 3,column=2)

        self.angle_list_entry = self._create_entry("Angle List:", 2,column=2,frame=self.experimental_frame)


    def _create_entry(self, text, row,column=1,sticky=tk.W,frame=None):
        if(frame==None):
            frame=self.main_frame

        label = ttk.Label(frame, text=text)
        label.grid(row=row, column=0, sticky=sticky, padx=5, pady=5)
        entry = ttk.Entry(frame)
        entry.grid(row=row, column=column, padx=5, pady=5)
        return entry

    def _create_combobox(self, frame, text, values, row, command=None,width=18):
        label = ttk.Label(frame, text=text)
        label.grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
        combobox = ttk.Combobox(frame, values=values, state="readonly",width=width)
        combobox.grid(row=row, column=1, padx=5, pady=5)
        if command:
            combobox.bind("<<ComboboxSelected>>", command)
        return combobox
    
    def _create_comboboxes(self):
            self.measurement_type_combobox = self._create_combobox(self.main_frame, "Measurement Type:", ["Current Source Measurement", "Voltage Bias Measurement"], 2, command=self.on_measurement_type_change)
            self.measurement_type_combobox.bind("<<ComboboxSelected>>", self.update_measurement_recipe_options)
            
            self.probe_number_combobox = self._create_combobox(self.main_frame, "Number of Probes:", [2, 4], 3)
            self.probe_number_combobox.bind("<<ComboboxSelected>>", self.update_measurement_recipe_options)


            self.pin_number_combobox = self._create_combobox(self.main_frame, "Numbers of Pins:", [1,24,40], 4)

            # Experimental setup comboboxes
            self.magnetic_coil_combobox = self._create_combobox(self.experimental_frame, "Magnetic Coil:", ["Option1", "Option2"], 0)
            self.magnetic_coil_combobox.bind("<<ComboboxSelected>>", self.update_measurement_recipe_options)

            self.cannon_board_combobox = self._create_combobox(self.experimental_frame, "Cannon Board:", ["Option1", "Option2"], 1)
            self.cannon_board_combobox.bind("<<ComboboxSelected>>", self.update_measurement_recipe_options)

            self.goniometer_combobox = self._create_combobox(self.experimental_frame, "Goniometer:", ["Option1", "Option2"], 2)
            self.goniometer_combobox.bind("<<ComboboxSelected>>", self.update_measurement_recipe_options)

            self.current_dynamic_combobox_1 = self._create_combobox(self.experimental_frame, "Current Source:", ["Option1", "Option2"], 3)
            self.current_dynamic_combobox_1.bind("<<ComboboxSelected>>", self.update_measurement_recipe_options)

            self.voltage_dynamic_combobox_2 = self._create_combobox(self.experimental_frame, "Voltage Measure:", ["Option1", "Option2"], 4)
            self.voltage_dynamic_combobox_2.bind("<<ComboboxSelected>>", self.update_measurement_recipe_options)

            # Measurement Recipe Combobox
            self.measurement_recipe_combobox = self._create_combobox(self.main_frame, "Measurement Recipe:", self.get_measurement_recipe_names(), 9,width=30)
            self.measurement_recipe_combobox.bind("<<ComboboxSelected>>", self.on_measurement_recipe_change)
    
    def get_measurement_recipe_names(self):
        df = Measurements_available()
        return df["Measurement_name"].tolist()


    def _create_buttons(self):
        self.folder_button = ttk.Button(self.main_frame, text="Select Save Folder", command=self.select_folder)
        self.folder_button.grid(row=8, column=0, pady=10)

        self.start_button = ttk.Button(self.main_frame, text="Start Measurement", command=self.start_measurement)
        self.start_button.grid(row=11, column=0, columnspan=2, pady=20)

        self.update_setups_button = ttk.Button(self.main_frame, text="Update Setups", command=self.update_comboboxes)
        self.update_setups_button.grid(row=10, column=0, columnspan=2, pady=10)

    def _create_labels(self):
        self.current_value_label = ttk.Label(self.main_frame, text="Current Value:")
        self.current_value_label.grid(row=5, column=0, sticky=tk.W, padx=5, pady=5)

        self.voltage_limit_label = ttk.Label(self.main_frame, text="Voltage Limit:")
        self.voltage_limit_label.grid(row=6, column=0, sticky=tk.W, padx=5, pady=5)

        self.channel_list_label = ttk.Label(self.main_frame, text="Cannon Board Channels:")
        self.channel_list_label.grid(row=2, column=2, sticky=tk.W, padx=5, pady=5)

        self.angle_list_label = ttk.Label(self.experimental_frame, text="Angle List:")
        self.angle_list_label.grid(row=1, column=2, sticky=tk.N, padx=5, pady=5)

        self.folder_path_label = ttk.Label(self.main_frame, text="No folder selected")
        self.folder_path_label.grid(row=8, column=1, pady=10, sticky=tk.W)
    
    def _create_checkboxes(self):
        self.filter_params_var = tk.IntVar()
        self.filter_setup_var = tk.IntVar()

        self.filter_params_checkbox = ttk.Checkbutton(self.main_frame, text="Filter by Parameters", variable=self.filter_params_var, command=self.update_measurement_recipe_options)
        self.filter_params_checkbox.grid(row=7, column=0, sticky=tk.W, padx=5, pady=5)

        self.filter_setup_checkbox = ttk.Checkbutton(self.main_frame, text="Filter by Setup", variable=self.filter_setup_var, command=self.update_measurement_recipe_options)
        self.filter_setup_checkbox.grid(row=7, column=1, sticky=tk.W, padx=5, pady=5)
        

    def update_measurement_recipe_options(self,event=0):
        df = Measurements_available()

        # If the Filter by Parameters checkbox is selected
        if self.filter_params_var.get():
            # Filter the dataframe based on the compatible parameters
            if not df.empty:
                df = df[df["Compatible Measurement Type"].apply(lambda x: self.measurement_type_combobox.get() in x)]
            if not df.empty:
                df = df[df["Compatible Probe Number"].apply(lambda x: int(self.probe_number_combobox.get()) in x)]
            

        # If the Filter by Setup checkbox is selected
        if self.filter_setup_var.get():
            if not df.empty:
                df = df[df["Compatible Magnetic Coil"].apply(lambda x: self.magnetic_coil_combobox.get() in x)]
            if not df.empty:
                df = df[df["Compatible Current Source"].apply(lambda x: self.current_dynamic_combobox_1.get() in x)]
            if not df.empty:
                df = df[df["Compatible Voltage Source"].apply(lambda x: self.voltage_dynamic_combobox_2.get() in x)]
            if not df.empty:
                df = df[df["Compatible CannonBoard"].apply(lambda x: self.cannon_board_combobox.get() in x)]
            if not df.empty:
                df = df[df["Compatible Goniometer"].apply(lambda x: self.goniometer_combobox.get() in x)]
            print("HI")
        
        if(self.measurement_recipe_combobox.get() not in df["Measurement_name"].tolist()):
            self.measurement_recipe_combobox.set('')
        # If the dataframe is not empty after all the filtering, update the combobox
        if not df.empty:
            self.measurement_recipe_combobox["values"] = df["Measurement_name"].tolist()
        else:
            # If the dataframe is empty, then clear the options
            self.measurement_recipe_combobox["values"] = []

    def on_measurement_recipe_change(self, event):
        # Get the currently selected recipe
        selected_recipe = self.measurement_recipe_combobox.get()
        if not selected_recipe:
            return

        df = Measurements_available()
        recipe_df = df[df["Measurement_name"] == selected_recipe]
        if recipe_df.empty:
            return

        # Assuming each measurement recipe will only have one row in the dataframe
        recipe_info = recipe_df.iloc[0]

        # Update Measurement Type combobox
        compatible_measurement_types = recipe_info["Compatible Measurement Type"]
        if compatible_measurement_types:
            self.measurement_type_combobox["values"] = compatible_measurement_types
            self.measurement_type_combobox.set(compatible_measurement_types[0])

        # Update Number of Probes combobox (Adjust based on your dataframe structure)
        compatible_probe_number = recipe_info["Compatible Probe Number"]
        if compatible_measurement_types:
            self.probe_number_combobox["values"] = compatible_probe_number
            self.probe_number_combobox.set(compatible_probe_number[0])

        # Update comboboxes for experimental setups based on the chosen recipe
        available_setups_df = self.get_available_setups()  # Function that returns available setups dataframe

        # Magnetic Coil
        compatible_coils = recipe_info["Compatible Magnetic Coil"]
        available_coils = available_setups_df[available_setups_df["Type"] == "Magnetic coil"]["Nickname"].tolist()
        intersection_coils = list(set(compatible_coils) & set(available_coils))
        self.magnetic_coil_combobox["values"] = intersection_coils
        if intersection_coils and self.magnetic_coil_combobox.get() not in intersection_coils:
            self.magnetic_coil_combobox.set(intersection_coils[0])

        
        # Current Source
        compatible_current_sources = recipe_info["Compatible Current Source"]
        available_current_sources = available_setups_df[(available_setups_df["Type"] == "Current Source") | (available_setups_df["Type"] == "Sourcemeter")]["Nickname"].tolist()
        intersection_current_sources = list(set(compatible_current_sources) & set(available_current_sources))
        self.current_dynamic_combobox_1["values"] = intersection_current_sources
        if intersection_current_sources and self.current_dynamic_combobox_1.get() not in intersection_current_sources:
            self.current_dynamic_combobox_1.set(intersection_current_sources[0])

        # Voltage Source
        compatible_voltage_sources = recipe_info["Compatible Voltage Source"]
        available_voltage_sources = available_setups_df[(available_setups_df["Type"] == "Voltage Source") | (available_setups_df["Type"] == "Sourcemeter")]["Nickname"].tolist()
        intersection_voltage_sources = list(set(compatible_voltage_sources) & set(available_voltage_sources))
        self.voltage_dynamic_combobox_2["values"] = intersection_voltage_sources
        if intersection_voltage_sources and self.voltage_dynamic_combobox_2.get() not in intersection_voltage_sources:
            self.voltage_dynamic_combobox_2.set(intersection_voltage_sources[0])

        # Cannon Board
        compatible_cannon_boards = recipe_info["Compatible CannonBoard"]
        available_cannon_boards = available_setups_df[available_setups_df["Type"] == "Cannon Board"]["Setup_Name"].tolist()
        intersection_cannon_boards = list(set(compatible_cannon_boards) & set(available_cannon_boards))
        self.cannon_board_combobox["values"] = available_setups_df[available_setups_df["Type"] == "Cannon Board"]["Nickname"].tolist()
        if intersection_cannon_boards and self.cannon_board_combobox.get() not in available_setups_df[available_setups_df["Type"] == "Cannon Board"]["Nickname"].tolist():
            self.cannon_board_combobox.set(available_setups_df[available_setups_df["Type"] == "Cannon Board"]["Nickname"].tolist()[0])

        # Goniometer
        compatible_goniometers = recipe_info["Compatible Goniometer"]
        available_goniometers = available_setups_df[available_setups_df["Type"] == "Goniometer"]["Nickname"].tolist()
        intersection_goniometers = list(set(compatible_goniometers) & set(available_goniometers))
        self.goniometer_combobox["values"] = intersection_goniometers
        if intersection_goniometers and self.goniometer_combobox.get() not in intersection_goniometers:
            self.goniometer_combobox.set(intersection_goniometers[0])
        
    def get_available_setups(self):
        # Sample function to get dataframe (Modify this as per your needs)
        df_available_setups=External_Instruments.recognize_setup()
        wmi = win32com.client.GetObject("winmgmts:")
        for usb in wmi.InstancesOf("Win32_SerialPort"):
            if("Arduino" in usb.Description):
                new_row={"Setup_Name" : "Arduino", "GPIB_ADDRESS": usb.DeviceID, "IDN_RESPONSE":"?","Type":"Current Invertor","Nickname":usb.DeviceID}
                #df_available_setups = pd.concat([df_available_setups,new_row],ignore_index=True)
                new_row_df = pd.DataFrame([new_row])  # Convert new_row to a DataFrame
                df_available_setups = pd.concat([df_available_setups, new_row_df], ignore_index=True)


            else:
                new_row={"Setup_Name" : "Cannon Board", "GPIB_ADDRESS": usb.DeviceID, "IDN_RESPONSE":"?","Type":"Cannon Board","Nickname":usb.DeviceID}
                new_row_df = pd.DataFrame([new_row])  # Convert new_row to a DataFrame
                df_available_setups = pd.concat([df_available_setups, new_row_df], ignore_index=True)


        
        new_row={"Setup_Name" : "Goniometer", "GPIB_ADDRESS": "don't worry", "IDN_RESPONSE":"?","Type":"Goniometer","Nickname":"Goniometer"}
        new_row_df = pd.DataFrame([new_row])  # Convert new_row to a DataFrame
        df_available_setups = pd.concat([df_available_setups, new_row_df], ignore_index=True)



        new_row={"Setup_Name" : "None", "GPIB_ADDRESS": "don't worry", "IDN_RESPONSE":"?","Type":"Goniometer","Nickname":"None"}
        new_row_df = pd.DataFrame([new_row])  # Convert new_row to a DataFrame
        df_available_setups = pd.concat([df_available_setups, new_row_df], ignore_index=True)



        

        df_available_setups=df_available_setups.dropna()
        #df_available_setups=df_available_setups[df_available_setups["GPIB_ADDRESS"]!="?"]

        if df_available_setups[df_available_setups["Setup_Name"] == "Arduino"].empty and not df_available_setups[df_available_setups["Setup_Name"] == "HP6032A"].empty:
            print("Hey")
            df_available_setups = df_available_setups.loc[~(df_available_setups["Setup_Name"] == "HP6032A")]


        
        
        return df_available_setups



    def on_measurement_type_change(self, event):
        measurement_type = self.measurement_type_combobox.get()

        if measurement_type == "Current Source Measurement":
            self.current_value_label.config(text="Current Value:")
            self.voltage_limit_label.config(text="Voltage Limit:")
            ttk.Label(self.experimental_frame, text="Current Source:").grid(row=3, column=0, sticky=tk.W, padx=5, pady=5)
            ttk.Label(self.experimental_frame, text="Voltage Measure:").grid(row=4, column=0, sticky=tk.W, padx=5, pady=5)
        else:
            self.current_value_label.config(text="Voltage Bias Value:")
            self.voltage_limit_label.config(text="Current Limit:")
            ttk.Label(self.experimental_frame, text="Current Measure:").grid(row=3, column=0, sticky=tk.W, padx=5, pady=5)
            ttk.Label(self.experimental_frame, text="Voltage Bias Source:").grid(row=4, column=0, sticky=tk.W, padx=5, pady=5)

    def select_folder(self):
        self.save_directory = filedialog.askdirectory()
        if self.save_directory:
            self.folder_path_label.config(text=self.save_directory)
    
    def update_comboboxes(self):
        # Sample function to get dataframe (Modify this as per your needs)
        df_setup_test = self.get_available_setups()

        # Update the Magnetic Coil combobox
        magnetic_coil_options = df_setup_test[df_setup_test["Type"] == "Magnetic coil"]["Nickname"].tolist()
        self.magnetic_coil_combobox["values"] = magnetic_coil_options

        # Update the current_source combobox
        current_source_options = df_setup_test[(df_setup_test["Type"] == "Current Source") | (df_setup_test["Type"] == "Sourcemeter")]["Nickname"].tolist()
        self.current_dynamic_combobox_1["values"] = current_source_options

        # Update the voltage_source combobox
        voltage_source_options = df_setup_test[(df_setup_test["Type"] == "Voltage Source") | (df_setup_test["Type"] == "Sourcemeter")]["Nickname"].tolist()
        self.voltage_dynamic_combobox_2["values"] = voltage_source_options

        # Update the voltage_source combobox
        pcb_cannon_options = df_setup_test[(df_setup_test["Type"] == "Cannon Board")]["Nickname"].tolist()
        self.cannon_board_combobox["values"] = pcb_cannon_options

        goniometer_options = df_setup_test[(df_setup_test["Type"] == "Goniometer")]["Nickname"].tolist()
        self.goniometer_combobox["values"] = goniometer_options

        # Add similar code for other comboboxes if necessary

        # Call the function again after a certain delay (e.g., 5000 ms = 5 seconds)
        #self.after(5000, self.update_comboboxes)
    
    def connect_setups(self):
        available_setups = self.get_available_setups()

        current_setup_name=self.current_dynamic_combobox_1.get()
        voltage_setup_name=self.voltage_dynamic_combobox_2.get()

        pcb_setup_name=self.cannon_board_combobox.get()
        magnet_setup_name=self.magnetic_coil_combobox.get()
        goniometer_setup_name=self.goniometer_combobox.get()

        #setup_list=[current_setup,voltage_setup,pcb_setup,magnet_setup]


        if(available_setups[ (available_setups["Nickname"] == current_setup_name) ] ["Type"].values[0]== "Sourcemeter"):
          
           adress=available_setups[ (available_setups["Nickname"] == current_setup_name) ] ["GPIB_ADDRESS"].values[0]
           name=available_setups[ (available_setups["Nickname"] == current_setup_name) ] ["Setup_Name"].values[0]
           
           self.current_setup = External_Instruments.SourceMeter(baud_rate=57600,name=name, inst_address=adress)
           sorcemeter=True

        if(available_setups[ (available_setups["Nickname"] == voltage_setup_name) ] ["Type"].values[0]== "Sourcemeter"):
           adress=available_setups[ (available_setups["Nickname"] == voltage_setup_name) ] ["GPIB_ADDRESS"].values[0]
           name=available_setups[ (available_setups["Nickname"] == voltage_setup_name) ] ["Setup_Name"].values[0]
           if sorcemeter==False:
               self.voltage_setup = External_Instruments.SourceMeter(baud_rate=57600,name=name, inst_address=adress)
           else:
               self.voltage_setup=self.current_setup
        
        if(available_setups[ (available_setups["Nickname"] == magnet_setup_name) ] ["Type"].values[0] == "Magnetic coil"):
            
            adress=available_setups[ (available_setups["Nickname"] == magnet_setup_name) ] ["GPIB_ADDRESS"].values[0]
            name=available_setups[ (available_setups["Nickname"] == magnet_setup_name) ] ["Setup_Name"].values[0]
            arduino_port=False
            
            if not available_setups[available_setups["Setup_Name"] == "Arduino"].empty:
                arduino_port=available_setups[available_setups["Setup_Name"] == "Arduino"]["GPIB_ADDRESS"].values[0]
            self.magnet_setup = External_Instruments.Magnetic_Coils(name=name, inst_address=adress,setup=magnet_setup_name,arduino_port=arduino_port)

        if(available_setups[ (available_setups["Nickname"] == pcb_setup_name) ] ["Type"].values[0] == "Cannon Board"):
            adress=available_setups[ (available_setups["Nickname"] == pcb_setup_name) ] ["GPIB_ADDRESS"].values[0]
            self.cannon_setup=PCB_Board.Multiplexer_Board(offline=False,rest_time=0,port=adress)

        if(available_setups[ (available_setups["Nickname"] == goniometer_setup_name) ] ["Setup_Name"].values[0] == "Goniometer"):
            self.goniometer_setup=Goniometer.Goniometer()
        else:
            self.goniometer_setup=False
            

                        
            

    def start_measurement(self):
        
        H_list = generate_values(self.magnetic_field_range_entry.get(),self.increment_steps_entry.get())
        if(H_list==None):
            return -1
        
        available_pins=int(self.pin_number_combobox.get())
        measurement_type = self.measurement_type_combobox.get()
        number_of_probes = float(self.probe_number_combobox.get())
        current_value = float(self.current_value_entry.get())
        voltage_limit = float(self.voltage_limit_entry.get())
        channel_list=parse_input(self.channel_list_entry.get())
        channel_list=[int(channel) for channel in channel_list]
        angle_list=parse_input(self.angle_list_entry.get())

        self.connect_setups()

        recipe_name=self.measurement_recipe_combobox.get()

        Start_Recipe(recipe_name=recipe_name,channel_list=channel_list,available_pins=available_pins,current_source=self.current_setup,voltimeter=self.voltage_setup,goniometer=self.goniometer_setup,
                                                    magnet=self.magnet_setup,pcb_cannon=self.cannon_setup,save_file=self.save_directory,
                                           H_list=H_list,current=current_value,voltage_compliance=voltage_limit,angle_list=angle_list)



if __name__ == "__main__":
    app = InstrumentControlInterface()
    app.mainloop()


c:\Users\Pedro\anaconda3\lib\site-packages\pyvisa\ctwrapper\highlevel.py:226: VisaIOWarning: VI_WARN_CONFIG_NLOADED (1073676407): The specified configuration either does not exist or could not be loaded. VISA-specified defaults will be used.
  return self.handle_return_value(session, ret_value)  # type: ignore


In [6]:
a=0b11
b=0b10
a | b

3

In [7]:
int([0.0,1.0])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'